# [★★★★☆] Thin-walled Section For Frame Structures

In this example, we demonstrate how to create thin-walled arbitrary sections.

To represent conventional, bending moment only sections numerically, only geometry information, such as location and area, is required. This allows the usage of predefined basic shapes, such as rectangles, circles. And complex sections can be built up by using those basic shapes. However, such an approach is not practical for sections involving torsion and warping, as some important sectional properties can only be obtained by analysing the section as a whole. Hence, a general approach that only focuses on section discretisation is used. The discretisation refers to the division of the section into a number of fibres/cells. Those fibres are the smallest units that are used in numerical integration. The users shall provide the corresponding sectional properties.

To obtain necessary sectional properties, a section analysis shall be performed. For arbitrary sections, often a general purpose FEA based method shall be used. [This](https://doi.org/10.1002/9780470172667) book provides a thorough discussion on relevant numerical methods.

The [`sectionproperties`](https://sectionproperties.readthedocs.io/en/latest/) library implements such a method. It can be used to prepare data to be used in `suanPan`.

## Theory

We first discuss what sectional properties are required and provide a theoretical background.

The [`Cell3DOS`](../../../Library/Section/SectionOS/Cell3DOS.md) section has the following signature.

```text
section Cell3DOS (1) (2) (3) (4) (5) (6) [7] [8]
# (1) int, unique section tag
# (2) double, area
# (3) double, sectional coordiante
# (4) double, py
# (5) double, pz
# (6) int, material tag
# [7] double, eccentricity/location along y axis, default: 0.0
# [8] double, eccentricity/location along z axis, default: 0.0
```

Apart from the location (coordinates/eccentricities) and area, it requires sectional coordinate (warping function) and its derivatives.

The warping function is the conventional one that can be expressed as a function of locations.

$$
\omega=\omega(y,z).
$$

The integration shall result in a net-zero warping.

$$
\int_A\omega(y,z)\mathrm{d}A=0.
$$

Its derivatives along two axes are also required.

$$
\dfrac{\partial\omega}{\partial{}y},\qquad\dfrac{\partial\omega}{\partial{}z}.
$$

[Alemdar's thesis](http://hdl.handle.net/1853/22220) uses $$\gamma=2n\phi'$$ which is suitable for thin-walled open sections.
Here we use general formulation instead for complex, universal, arbitrary sections.
Interested readers can refer to [Pilkey's book](https://doi.org/10.1002/9780470172667) for more details.

## A Simple Example

We start with a simple circular example. Let's assume it is centred and has a diameter of $$10$$.

We use built-in functionality to create such a geometry and assign a mesh size of $$0.1$$.

In [ ]:
import sectionproperties.pre.library.primitive_sections as sections
from sectionproperties.analysis.section import Section

# approximate a circle by a polygon
geometry = sections.circular_section(d=10, n=64)
geometry.create_mesh(mesh_sizes=[0.1])
section = Section(geometry)
section.plot_mesh(title="Circular Section Mesh")

The mesh is generated internally using second-order triangular elements (CPS6 in ABAQUS notation).
One can check the details of the created elements.
The following shows the first element.

In [ ]:
ele = section.elements[0]
print(ele)

### Area

The library has implemented basic utility functions to elements, such that basic properties for each element can be conveniently obtained.

In [ ]:
print("(area, qx, qy, ixx, iyy, ixy, e, g, rho)")
print(ele.geometric_properties())

### Locations

We assume the triangle element is small enough so that the centroid can represent the location of the element. The centroid can be obtained by averaging the coordinates of the first three nodes.

In [ ]:
coords = ele.coords[:, :3]
centroid = coords.mean(axis=1)
print(centroid)

### Warping Function

The library provides a function to calculate the warping function.

In [ ]:
from sectionproperties.analysis import solver

stiffness, force = section.assemble_torsion()
omega = solver.solve_direct_lagrange(stiffness, force)
print(omega)
print(f"sum(omega) = {omega.sum()}")

As we are using a circular section as the example, the warping function is zero everywhere.
The above method computes the warping in the local coordinate system.
If one wants to use the centroid as the origin, one shall translate the coordinates first.

### Partial Derivatives

To obtain $$\dfrac{\partial\omega}{\partial{}y}$$ and $$\dfrac{\partial\omega}{\partial{}z}$$, we use the shape functions.

In [ ]:
from sectionproperties.analysis.fea import shape_function

N, B, j = shape_function(ele.coords, [0, 1.0 / 3.0, 1.0 / 3.0, 1.0 / 3.0])
centre_omega = omega[ele.node_ids].dot(N)
centre_partial = B @ omega[ele.node_ids]
print(f"warping function = {centre_omega}")
print(f"derivatives = {centre_partial}")

Since for circular sections, there is no warping, the corresponding derivatives are trivial.

### A Universal Function

It appears to be easy to obtain necessary properties for each element. Here we provide a universal function to generate the required data.

One shall adjust it accordingly for specific purposes.

In [ ]:
from numpy import ndarray, array
from sectionproperties.analysis.fea import Tri6


class Cell3DOS:
    tag: int
    area: float
    y: float
    z: float
    omega: float
    py: float
    pz: float

    def __init__(self, ele: Tri6, omega: ndarray = None):
        """
        If `omega` is None, no warping is considered.
        """

        N, B, self.area = shape_function(ele.coords, array([0, 1.0 / 3.0, 1.0 / 3.0, 1.0 / 3.0]))
        self.tag = ele.el_id + 1  # el_id is zero-based
        self.y, self.z = ele.coords[:, :3].mean(axis=1)
        self.omega = omega.dot(N) if omega is not None else 0
        self.py, self.pz = (B @ omega) if omega is not None else (0, 0)

    def export(self):
        return (
            f"section Cell3DOS"
            f" {self.tag}"
            f" {self.area:+E}"
            f" {self.omega:+E}"
            f" {self.py:+E}"
            f" {self.pz:+E}"
            # f" $mat_tag"
            f" 1"
            f" {self.y:+E}"
            f" {self.z:+E}"
        )


def to_cell3dos(geometry, *, no_warping: bool = False):
    section = Section(geometry)
    if no_warping:
        return [Cell3DOS(ele) for ele in section.elements]

    # section.calculate_geometric_properties()
    # for ele in section.elements:
    #     ele.coords[0, :] -= section.section_props.cx
    #     ele.coords[1, :] -= section.section_props.cy
    stiffness, force = section.assemble_torsion()
    omega = solver.solve_direct_lagrange(stiffness, force)

    return [Cell3DOS(ele, omega[ele.node_ids]) for ele in section.elements]

Now it is possible to pass in a geometry, and call the export function to generate cell data.

In [ ]:
cell3dos = to_cell3dos(geometry)

with open("cell3dos.sp", "w") as f:
    for cell in cell3dos:
        f.write(cell.export() + "\n")

    f.write(f"section Fibre3DOS {1 + len(cell3dos)} ")
    f.write(" ".join([str(cell.tag) for cell in cell3dos]) + "\n")

### Validation

Now it is possible to create a simple single element model to validate the data. Remember to change the material tag accordingly.

```text
node 1 0 0 0
node 2 1 0 0
material ElasticOS 1 2. 0. ! <--- so that G=1
file cell3dos.sp
orientation B3DOSL 1 0. 0. 1.
element B31OS 1 1 2 1203 1 6 ! <--- 1203 is the tag for Fibre3DOS section
fix2 1 E 1
displacement 1 0 1E-3 4 2
step static 1
set ini_step_size 1E-1
set fixed_step_size true
analyze
peek node 2
exit
```

The nodal displacement and force are as follows.

```text
Node 2:
Coordinate:
  1.0000e+00  0.0000e+00  0.0000e+00
Displacement:
 -6.2326e-06  7.6478e-12  4.5576e-11  1.0000e-03 -9.1148e-11  1.5311e-11  1.0000e-03
Resistance:
 -3.2660e-19 -2.6131e-20 -1.3261e-19  9.7745e-01  6.2923e-19 -8.1782e-20  4.8572e-17
```

The circular section has a torsional constant as follows.

$$
J=\dfrac{1}{2}\pi{}r^4=\dfrac{1}{2}\pi\left(\dfrac{10}{2}\right)^4=981.7.
$$

This is about the numerical result $$977.5$$.

## W-Section Example

In this example, we manually create a W section. It corresponds to the `W36X330` section.

In [ ]:
from sectionproperties.pre.geometry import Geometry

bf = 16.6
tw = 1.02
tf = 1.85
d = 37.7

y_offset = 0
z_offset = 0

points = [
    [-bf / 2, -d / 2],
    [bf / 2, -d / 2],
    [bf / 2, -d / 2 + tf],
    [tw / 2, -d / 2 + tf],
    [tw / 2, d / 2 - tf],
    [bf / 2, d / 2 - tf],
    [bf / 2, d / 2],
    [-bf / 2, d / 2],
    [-bf / 2, d / 2 - tf],
    [-tw / 2, d / 2 - tf],
    [-tw / 2, -d / 2 + tf],
    [-bf / 2, -d / 2 + tf],
]

for p in points:
    p[0] += y_offset
    p[1] += z_offset

facets = [
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [5, 6],
    [6, 7],
    [7, 8],
    [8, 9],
    [9, 10],
    [10, 11],
    [11, 0],
]

holes = []
control_points = [[y_offset, z_offset]]
geometry = Geometry.from_points(points, facets, control_points)

geometry.create_mesh(mesh_sizes=.2)
geometry.plot_geometry()

cell3dos = to_cell3dos(geometry, no_warping=False)

with open("w36x330.sp", "w") as f:
    for cell in cell3dos:
        f.write(cell.export() + "\n")

    f.write(f"section Fibre3DOS {1 + len(cell3dos)} ")
    f.write(" ".join([str(cell.tag) for cell in cell3dos]) + "\n")